In [ ]:
# ANGELO DAVID PADILLA ROMERO_BOOTCAMP-IA_NIVEL-AVANZADO_TALENTOTECH - INSTRUCTOR: CRISTIAN CAMILO TIRADO CIFUENTES - 2024

# Instalación de Librerias
!pip install scikit-learn xgboost pandas numpy matplotlib seaborn kaggle requests

In [ ]:
# Importación de Librerias
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report
from xgboost import XGBClassifier, XGBRegressor
import matplotlib.pyplot as plt
import subprocess
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Descargar el dataset desde Kaggle
kaggle_url = "mssmartypants/rice-type-classification"
file_name = "riceClassification.csv"
subprocess.run(['kaggle', 'datasets', 'download', '-d', kaggle_url, '-f', file_name])

CompletedProcess(args=['kaggle', 'datasets', 'download', '-d', 'mssmartypants/rice-type-classification', '-f', 'riceClassification.csv'], returncode=0)

In [ ]:
# Descomprimir el Archivo
!unzip riceClassification.csv.zip

Archive:  riceClassification.csv.zip
  inflating: riceClassification.csv  


In [ ]:
# Carga y Procesamiento de Datos
data = pd.read_csv('riceClassification.csv')
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18185 entries, 0 to 18184
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               18185 non-null  int64  
 1   Area             18185 non-null  int64  
 2   MajorAxisLength  18185 non-null  float64
 3   MinorAxisLength  18185 non-null  float64
 4   Eccentricity     18185 non-null  float64
 5   ConvexArea       18185 non-null  int64  
 6   EquivDiameter    18185 non-null  float64
 7   Extent           18185 non-null  float64
 8   Perimeter        18185 non-null  float64
 9   Roundness        18185 non-null  float64
 10  AspectRation     18185 non-null  float64
 11  Class            18185 non-null  int64  
dtypes: float64(8), int64(4)
memory usage: 1.7 MB


,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
count,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000
mean,9093.000000,7036.492989,151.680754,59.807851,0.915406,7225.817872,94.132952,0.616653,351.606949,0.707998,2.599081,0.549079
std,5249.701658,1467.197150,12.376402,10.061653,0.030575,1502.006571,9.906250,0.104389,29.500620,0.067310,0.434836,0.497599
min,1.000000,2522.000000,74.133114,34.409894,0.676647,2579.000000,56.666658,0.383239,197.015000,0.174590,1.358128,0.000000
25%,4547.000000,5962.000000,145.675910,51.393151,0.891617,6125.000000,87.126656,0.538530,333.990000,0.650962,2.208527,0.000000
50%,9093.000000,6660.000000,153.883750,55.724288,0.923259,6843.000000,92.085696,0.601194,353.088000,0.701941,2.602966,1.000000
75%,13639.000000,8423.000000,160.056214,70.156593,0.941372,8645.000000,103.559146,0.695664,373.003000,0.769280,2.964101,1.000000
max,18185.000000,10210.000000,183.211434,82.550762,0.966774,11008.000000,114.016559,0.886573,508.511000,0.904748,3.911845,1.000000


In [ ]:
# Separación de Características y Objetivo
X = data.drop(columns=['Class', 'id'])  # Eliminas 'Class' e 'id' de las características
y = data['Class']  # La variable objetivo es 'Class'

In [ ]:
# Dividir los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Escalado de los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Regresión Logistica + Grid Search
param_grid_logreg = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga', 'lbfgs']
}
grid_search_logreg = GridSearchCV(LogisticRegression(), param_grid_logreg, cv=5)
grid_search_logreg.fit(X_train, y_train)

print(f"Mejores hiperparámetros para Logistic Regression: {grid_search_logreg.best_params_}")
print(f"Mejor puntuación en entrenamiento: {grid_search_logreg.best_score_}")

Mejores hiperparámetros para Logistic Regression: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Mejor puntuación en entrenamiento: 0.9897578908447418


In [ ]:
# Predicciones y Evaluación
y_pred_class = grid_search_logreg.predict(X_test)
print(classification_report(y_test, y_pred_class))
print(f"Accuracy: {accuracy_score(y_test, y_pred_class)}")

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1632
           1       0.99      0.99      0.99      2005

    accuracy                           0.99      3637
   macro avg       0.99      0.99      0.99      3637
weighted avg       0.99      0.99      0.99      3637

Accuracy: 0.9898267803134452


In [ ]:
# XGBoost + Random Search
param_dist_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]
}
random_search_xgb = RandomizedSearchCV(XGBClassifier(), param_distributions=param_dist_xgb, n_iter=10, cv=5, random_state=42)
random_search_xgb.fit(X_train, y_train)

# Evaluación de XGBoost
print(f"Mejores hiperparámetros para XGBoost: {random_search_xgb.best_params_}")
print(f"Mejor puntuación en entrenamiento: {random_search_xgb.best_score_}")
y_pred_class = random_search_xgb.predict(X_test)
print(classification_report(y_test, y_pred_class))
print(f"Accuracy: {accuracy_score(y_test, y_pred_class)}")


Mejores hiperparámetros para XGBoost: {'subsample': 1, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.8}
Mejor puntuación en entrenamiento: 0.9899639346547449
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1632
           1       0.99      0.99      0.99      2005

    accuracy                           0.99      3637
   macro avg       0.99      0.99      0.99      3637
weighted avg       0.99      0.99      0.99      3637

Accuracy: 0.9898267803134452


In [ ]:
# Comparación entre modelos -Mejores parámetros y puntuaciones-
print("Best parameters (Logistic Regression):", grid_search_logreg.best_params_)
print("Best score (Logistic Regression):", grid_search_logreg.best_score_)
print("\nBest parameters (XGBoost):", random_search_xgb.best_params_)
print("Best score (XGBoost):", random_search_xgb.best_score_)

Best parameters (Logistic Regression): {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best score (Logistic Regression): 0.9897578908447418

Best parameters (XGBoost): {'subsample': 1, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.8}
Best score (XGBoost): 0.9899639346547449
